In [103]:
import os
import nengo
import nengo_dl
import numpy as np
import keras
import tensorflow as tf
import sklearn.model_selection
from tensorflow.python.keras import Model, Sequential, Input
from tensorflow.python.keras.layers import Conv2D, BatchNormalization, Dropout, AveragePooling2D, Flatten, Dense


In [104]:
dataset = np.load(file=os.path.join('..', 'datasets', 'output', 'p300-target-nontarget.npz'))
target_x, non_target_x = dataset['target_features'], dataset['non_target_features']

# trim target or non target data to have same amount of samples
if target_x.shape[0] > non_target_x.shape[0]:
    target_x = target_x[:non_target_x.shape[0]]
else:
    non_target_x = non_target_x[:target_x.shape[0]]

print('target_x:', target_x.shape, 'non_target_x:', non_target_x.shape)

# (1, 0) vector for target data
target_y = np.tile(np.array([1, 0]), (target_x.shape[0], 1))
# (0, 1) vector for non target data
non_target_y = np.tile(np.array([0, 1]), (non_target_x.shape[0], 1))
print('target_y:', target_y, 'non_target_y:', non_target_y)

features = np.concatenate((target_x, non_target_x), axis=0)
labels = np.vstack((target_y, non_target_y))

features = features.reshape((features.shape[0], 1, -1))
labels = labels.reshape((features.shape[0], 1, -1))

print(features, labels)

target_x: (1525, 3, 1201) non_target_x: (1525, 3, 1201)
target_y: [[1 0]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]] non_target_y: [[0 1]
 [0 1]
 [0 1]
 ...
 [0 1]
 [0 1]
 [0 1]]
[[[-8.04011680e-06 -8.70710899e-06 -8.82429649e-06 ... -4.72189662e-05
   -4.79357631e-05 -4.91300990e-05]]

 [[ 7.30717020e-06  8.66459208e-06  9.64115458e-06 ... -5.55348987e-05
   -5.55178089e-05 -5.56154652e-05]]

 [[-3.07758085e-06 -3.97748319e-06 -4.96820585e-06 ... -4.10499796e-05
   -4.04259562e-05 -3.97355265e-05]]

 ...

 [[-4.32359564e-06 -5.23143256e-06 -6.16380561e-06 ... -7.87536075e-06
   -7.30163028e-06 -6.75841739e-06]]

 [[-4.43766762e-06 -3.97685219e-06 -3.12382485e-06 ... -2.93570295e-05
   -2.85925642e-05 -2.82599226e-05]]

 [[ 1.79280477e-06  4.33307705e-07 -6.83635655e-07 ... -6.05487653e-05
   -6.21936627e-05 -6.38095685e-05]]] [[[1 0]]

 [[1 0]]

 [[1 0]]

 ...

 [[0 1]]

 [[0 1]]

 [[0 1]]]


In [105]:
# global seed for consistency
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [106]:
def create_nn():
    input_layer = Input(shape=(3, 1201, 1))
    conv_input = Conv2D(name='conv1',filters=6, kernel_size=(3, 3), activation=tf.nn.elu)(input_layer)
    batch_norm1 = BatchNormalization()(conv_input)
    dropout1 = Dropout(0.5)(batch_norm1)
    avg_pooling = AveragePooling2D(pool_size=(1, 8))(dropout1)
    flatten = Flatten()(avg_pooling)
    dense1 = Dense(100, activation=keras.activations.elu)(flatten)
    batch_norm2 = BatchNormalization()(dense1)
    dropout2 = Dropout(0.5)(batch_norm2)
    output_layer = Dense(2, activation=keras.activations.softmax)(dropout2)

    return Model(inputs=input_layer, outputs=output_layer), input_layer, output_layer, conv_input


In [107]:
def run_spiking(
        activation, test_data_x, test_data_y, params_file='keras_to_snn_params',
        scale_firing_rates=1, synapse=None
):
    model, input_layer, output_layer, conv2d = create_nn()
    converter = nengo_dl.Converter(model, swap_activations={ tf.nn.elu: activation },
                                   scale_firing_rates=scale_firing_rates, synapse=synapse)

    nengo_input, nengo_output = converter.inputs[input_layer], converter.inputs[output_layer]

    with nengo_dl.Simulator(converter.net, minibatch_size=10, progress_bar=False) as nengo_sim:
        nengo_sim.load_params(params_file)
        data = nengo_sim.predict({ nengo_input: test_data_x })

    predictions = np.argmax(data[nengo_output][:, -1], axis=-1)
    acc = (predictions == test_data_y[:, 0, 0]).mean()
    print('accuracy: {:4f}%'.format(acc*100))
    return acc


def run_model(
        validation_metrics, test_metrics, snn_metrics,
        train_x, train_y, validation_x, validation_y, test_x, test_y, spike_rate_hz = 250
):
    model, input_layer, output_layer, conv1 = create_nn()

    converter = nengo_dl.Converter(model)

    nengo_input, nengo_output = converter.inputs[input_layer], converter.outputs[output_layer]

    with converter.net:
        output_probe = converter.outputs[output_layer]
        conv2D_probe = nengo.Probe(converter.layers[conv1])

    with nengo_dl.Simulator(converter.net, minibatch_size=16) as sim:
        sim.compile(
            optimizer=tf.optimizers.Adam(),
            loss={
                output_probe: tf.losses.BinaryCrossentropy(from_logits=True),
                conv2D_probe: tf.losses.mse
            },
            loss_weights={ output_probe: 1, conv2D_probe: 1e-3 }
        )

        hist = sim.fit(
            { nengo_input: train_x },
            {
                nengo_output: train_y,
                conv2D_probe: np.ones((train_y.shape[0], 1, conv2D_probe.size_in)) * spike_rate_hz
            },
            epochs=30,
            validation_data=(
                { nengo_input: validation_x },
                { nengo_output: validation_y }
            ),
            callbacks=[keras.callbacks.EarlyStopping(patience=5, verbose=1, restore_best_weights=True)],
            batch_size=16,
            shuffle=True
        )
        sim.save_params('./keras_to_snn_params')

        validation_metrics.append(hist)

        eval = sim.evaluate({ nengo_input: test_x }, { nengo_output: test_y })
        print(eval)
        test_metrics.append(eval)
        #
        # spiking_result = run_spiking(
        #     activation=nengo.SpikingRectifiedLinear(),
        #     test_data_x=test_x,
        #     test_data_y=test_y,
        #     synapse=0.01
        # )
        #
        # snn_metrics.append(spiking_result)


In [108]:
# Monte Carlo Cross Validation

# split data into 3/4 as training and 1/4 as testing
train_x, test_x, train_y, test_y = sklearn.model_selection.train_test_split(
    features, labels, test_size=0.25, random_state=seed
)

shuffle_split = sklearn.model_selection.ShuffleSplit(
    n_splits=30, test_size=0.25, random_state=seed
)

iter = 0
validation_metrics, test_metrics, snn_metrics = [], [], []
for training, validation in shuffle_split.split(train_x, train_y):
    run_model(
        validation_metrics=validation_metrics,
        test_metrics=test_metrics,
        snn_metrics=snn_metrics,
        train_x = train_x[training],
        train_y= train_y[training],
        validation_x= train_x[validation],
        validation_y= train_y[validation],
        test_x=test_x,
        test_y=test_y
    )
    iter += 1

print(validation_metrics, test_metrics, snn_metrics)

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/30
107/107 [==============================] - 2s 14ms/step - loss: 63.1864 - probe_loss: 0.7129 - probe_1_loss: 62473.5000 - val_loss: 0.7369 - val_probe_loss: 0.7369 - val_probe_1_loss: 0.0000e+00

C:\Users\itznu\anaconda3\envs\datascience-gpu\lib\site-packages\nengo_dl\converter.py:563: UserWarning: Activation type <function elu at 0x000001926233D0D8> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation
C:\Users\itznu\anaconda3\envs\datascience-gpu\lib\site-packages\nengo_dl\converter.py:309: UserWarning: Cannot convert BatchNormalization layer to native Nengo objects unless inference_only=True. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
C:\Users\itznu\anaconda3\envs\datascience-gpu\lib\site-packages\nengo_dl\converter.py:309: UserWarning: Layer type <class 'tensorflow.python.keras.layers.core.Dropout'> does not have a registered converter. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
C:\Users\itznu\anaconda3\envs\datascience-gpu\lib\site-packages\nengo_dl\converter.py:563: UserWarning: Activation type <function elu at 0x0000019264D00828> does not have a native 

KeyboardInterrupt: 